In [1]:
from geoband.API import *
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import geopandas as gpd
import pandas as pd
import json
from pandas import json_normalize
import pandas as pd
import folium
from folium.plugins import MarkerCluster
from geoband.API import *
import geopandas as gpd
import warnings
warnings.filterwarnings("ignore")
from shapely.geometry import Point, Polygon
import matplotlib.pyplot as plt
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster
from geopandas import GeoDataFrame
# GetCompasData로 주피터 노트북 경로에 파일 저장
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon, LineString
from fiona.crs import from_string

%matplotlib inline

import matplotlib.pyplot as plt
from libpysal.weights.contiguity import Queen
from libpysal import examples
import numpy as np
import pandas as pd
import geopandas as gpd
import os
import splot

In [2]:
#공간회귀분석

import pysal
import libpysal
from esda.moran import Moran
from esda.moran import Moran_Local
from libpysal.weights import Queen, Rook, KNN, DistanceBand, Kernel
from esda.getisord import G
from esda.moran import Moran
from spreg import OLS, ML_Lag, ML_Error, GM_Lag, GM_Error, TSLS

In [3]:
from matplotlib import font_manager, rc
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
plt.rc('font', family='NanumBarunGothic') #한글 폰트 지정?

In [4]:
suwon_112 = pd.read_csv('1.수원시_112신고이력(격자_관할경계매핑).csv')
suwon_112_grid = gpd.read_file('2.수원시_112신고(격자별 신고횟수).geojson')
others_112 = pd.read_csv('3.3개시도_112신고이력(관할경계매핑).csv')

### 112 신고이력 관련 데이터 가공
- 20개 이상의 사건종별 데이터를 6개의 중분류로 매핑해주기
- 교통 / 질서유지 / 중요범죄 / 기타범죄 / 기타_경찰업무 / 타기관_기타

In [5]:
def 사건종별_분류(x):
    교통 = ["교통사고","교통불편","교통위반"]
    질서유지 = ["기타경범","주취자","행패소란","보호조치","시비","위험방지","청소년비행","무전취식승차"]
    중요범죄 = ["절도","가정폭력","폭력"]
    기타범죄 = ["기타형사범","도박"]
    기타_경찰업무 = ["상담문의","분실습득","비상벨","실종(실종아동 등)"]
    타기관_기타 = ["기타_타기관","소음","서비스요청"]
    
    if x in 교통:
        return "교통"
    if x in 질서유지:
        return "질서유지"
    if x in 중요범죄:
        return "중요범죄"
    if x in 기타범죄:
        return "기타범죄"
    if x in 기타_경찰업무:
        return "기타_경찰업무"
    if x in 타기관_기타:
        return "타기관_기타"

In [6]:
def time_cut(x):
    if 7 >= x >= 2:
        return '새벽'
    elif 13 >= x >= 8:
        return '출근/점심시간'
    elif 19 >= x >= 14:
        return '오후/퇴근시간'
    elif 24 >= x >= 20:
        return '저녁/심야시간'
    elif x == 1:
        return '저녁/심야시간'
    elif x == 0:
        return '저녁/심야시간'

In [7]:
others_112 = pd.read_csv('3.3개시도_112신고이력(관할경계매핑).csv')
others_112.columns = ["사건종별","접수완료일자","접수완료시각","도착일시","관할치안센터","관할경찰서"]

suwon_112 = pd.read_csv('1.수원시_112신고이력(격자_관할경계매핑).csv')
suwon_112.columns = ["사건종별","접수완료일자","접수완료시각","도착일시","gid","관할치안센터","관할경찰서"]

In [8]:
def 신고이력데이터_처리(data):
    
    others_112 = data.copy()
    others_112["접수완료시각"] = others_112["접수완료시각"].str.replace("'","")
    others_112["접수완료일자"] = others_112["접수완료일자"].astype(str)
    #others_112["도착일시"] = others_112["도착일시"].astype(int).astype(str)
    # 도착일시가 NaN인 경우 존재 -> 도착일시가 NaN이 아닌 경우에 대해서만 출동 소요시각 구하기

    others_112["접수완료일시"] = others_112["접수완료일자"] + others_112["접수완료시각"]
    others_112["접수완료일시"] = pd.to_datetime(others_112["접수완료일시"],format="%Y%m%d%H%M%S")

    others_112["년"] = others_112["접수완료일시"].apply(lambda x : x.year)
    others_112["월"] = others_112["접수완료일시"].apply(lambda x : x.month)
    others_112["일"] = others_112["접수완료일시"].apply(lambda x : x.day)
    others_112["요일"] = others_112["접수완료일시"].apply(lambda x : x.weekday())
    others_112["시"] = others_112["접수완료일시"].apply(lambda x : x.hour)
    others_112["분"] = others_112["접수완료일시"].apply(lambda x : x.minute)
    others_112["초"] = others_112["접수완료일시"].apply(lambda x : x.second)

    # 도착일시가 정확히 기록된 경우와 그렇지 않은 경우로 나누기

    def get_도착일시(x):
        if len(str(x)) < 14:
            return np.NaN
        else:
            return str(int(float(x)))

    others_112["도착일시"] = others_112["도착일시"].apply(lambda x : get_도착일시(x))
    others_112["도착일시"] = pd.to_datetime(others_112["도착일시"],format="%Y%m%d%H%M%S")

    # 출동하는데 걸린 소요 시간
    others_112["소요시간"] = others_112["도착일시"] - others_112["접수완료일시"]

    # 사건종별 중분류
    others_112["사건종별_중분류"] = others_112["사건종별"].apply(lambda x : 사건종별_분류(x))
    
    # 시간대별 분류
    others_112["시간대"] = others_112["시"].apply(lambda x : time_cut(x))
    
    return others_112

In [9]:
suwon_112_get = 신고이력데이터_처리(suwon_112)

#others_112_get = 신고이력데이터_처리(others_112)

In [34]:
def get_barplot(data,x):
    groupby_data = data.groupby(x).agg({"발생건수":"sum"}).reset_index()
    sns.barplot(data = groupby_data
                , x=x, y="발생건수", palette='Blues_d')
    #return groupby_data

#### 수원시 격자별 112 신고데이터 가공완료
- 1.수원시_112신고이력(격자_관할경계매핑)_전처리완료.csv 내보내기

In [36]:
suwon_112_get.to_csv("1.수원시_112신고이력(격자_관할경계매핑)_전처리완료.csv",encoding="utf-8-sig",index=False)